# Customize DataLoader

This example shows **how to customize your own dataloader** for a new target dataset in **ZhiJian**.

Feel free to deploy model reusability technology on *any* dataset, with loading in the conventional `PyTorch` style.

+ Configure basic parameters, without dataset configuration:

In [5]:
from zhijian.trainers.base import prepare_args
from zhijian.models.utils import pprint, dict2args
training_mode = 'finetune'
args = dict2args({
    'dataset': 'Customized',
    'log_url': 'your/log/directory',                                           # log directory
    'model': 'timm.vit_base_patch16_224_in21k',                                # backbone network
    'config_blitz': "(LoRA.adapt): ...->(blocks[0:12].attn.qkv){inout1}->...", # addin blitz configuration
    'training_mode': training_mode,                                            # training mode
    'reuse_keys_blitz': "addin,head,fc_norm",                                  # reuse keys blitz configuration
    'optimizer': 'adam',                                                       # optimizer
    'batch_size': 64,                                                          # batch size
    'num_workers': 8,                                                          # num workers
    'max_epoch': 5,                                                            # max epoch
    'eta_min': 0,                                                              # eta_min of CosineAnnealingLR
    'lr': 1e-3,                                                                # learning rate
    'wd': 5e-5,                                                                # weight decay
    'gpu': '0',                                                                # gpu id
    'seed': 0,                                                                 # random seed
    'verbose': True,                                                           # control the verbosity of the output
    'only_do_test': False                                                      # test flag
})      

args = prepare_args(args, update_default=True)
pprint(vars(args))

{'aa': None,
 'addins': [{'hook': [['adapt', 'post']],
             'location': [['blocks', 0, 'attn', 'qkv']],
             'name': 'LoRA'},
            {'hook': [['adapt', 'post']],
             'location': [['blocks', 1, 'attn', 'qkv']],
             'name': 'LoRA'},
            {'hook': [['adapt', 'post']],
             'location': [['blocks', 2, 'attn', 'qkv']],
             'name': 'LoRA'},
            {'hook': [['adapt', 'post']],
             'location': [['blocks', 3, 'attn', 'qkv']],
             'name': 'LoRA'},
            {'hook': [['adapt', 'post']],
             'location': [['blocks', 4, 'attn', 'qkv']],
             'name': 'LoRA'},
            {'hook': [['adapt', 'post']],
             'location': [['blocks', 5, 'attn', 'qkv']],
             'name': 'LoRA'},
            {'hook': [['adapt', 'post']],
             'location': [['blocks', 6, 'attn', 'qkv']],
             'name': 'LoRA'},
            {'hook': [['adapt', 'post']],
             'location': [['blocks', 7, 'a

+ Set up the GPU and prepare the model:

In [6]:
import torch
import os
from zhijian.trainers.finetune import get_model

assert torch.cuda.is_available()
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
torch.cuda.set_device(int(args.gpu))

model, model_args, device = get_model(args)

/home/zhangyk/miniconda3/envs/zykycy/lib/python3.9/site-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name vit_base_patch16_224_in21k to current vit_base_patch16_224.augreg_in21k.
  model = create_fn(


+ Organize the custom dataset in the following structure: within the `your/dataset/dir` directory, create a separate folder for each category, and store all the data corresponding to each category within its respective folder:

    ```bash
    $ /your/dataset/directory
      ├── train
      │   ├── class_1
      │   │   ├── train_class_1_img_1.jpg
      │   │   ├── train_class_1_img_2.jpg
      │   │   ├── train_class_1_img_3.jpg
      │   │   └── ...
      │   ├── class_2
      │   │   ├── train_class_2_img_1.jpg
      │   │   └── ...
      │   ├── class_3
      │   │   └── ...
      │   ├── class_4
      │   │   └── ...
      │   ├── class_5
      │   │   └── ...
      └── test
          ├── class_1
          │   ├── test_class_1_img_1.jpg
          │   ├── test_class_1_img_2.jpg
          │   ├── test_class_1_img_3.jpg
          │   └── ...
          ├── class_2
          │   ├── test_class_2_img_1.jpg
          │   └── ...
          ├── class_3
          │   └── ...
          ├── class_4
          │   └── ...
          └── class_5
              └── ...
    ```

+ Set up the custom dataset:

In [11]:
import torchvision.transforms as transforms
from torchvision.datasets.folder import ImageFolder

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
    )
])
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
    )
])

train_dataset = ImageFolder(root='/data/zhangyk/data/custom/train', transform=train_transform)
val_dataset = ImageFolder(root='/data/zhangyk/data/custom/test', transform=val_transform)

+ Implement the corresponding loader:

In [12]:
train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=args.batch_size,
        num_workers=args.num_workers,
        pin_memory=True,
        shuffle=True
    )
val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=args.batch_size,
        num_workers=args.num_workers,
        pin_memory=True,
        shuffle=True
    )
num_classes = len(train_dataset.classes)

+ Set up the optimizer and the loss function:

In [13]:
import torch.optim as optim
import torch.nn as nn

optimizer = optim.Adam(
    model.parameters(),
    lr=args.lr,
    weight_decay=args.wd
    )
lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    args.max_epoch,
    eta_min=args.eta_min
    )
criterion = nn.CrossEntropyLoss()

+ Set up the trainer and start training and testing:

In [14]:
from zhijian.trainers.base import prepare_trainer
trainer = prepare_trainer(
    args,
    model=model, model_args=model_args, device=device,
    train_loader=train_loader,
    val_loader=val_loader,
    num_classes=num_classes,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    criterion=criterion
    )

trainer.fit()
trainer.test()

Log level set to: INFO
Log files are recorded in: your/log/directory/0718-20-10-57-792
Trainable/total parameters of the model: 0.30M / 86.10M (0.34700%)



      Epoch   GPU Mem.       Time       Loss         LR


        1/5      5.48G      1.686       1.73      0.001: 100%|██████████| 1.00/1.00 [00:01<00:00, 1.22s/batch]



      Epoch   GPU Mem.       Time      Acc@1      Acc@5


        1/5      5.48G     0.3243         16        100: 100%|██████████| 1.00/1.00 [00:00<00:00, 2.39batch/s]
***   Best results: [Acc@1: 16.0], [Acc@5: 100.0]


...


      Epoch   GPU Mem.       Time      Acc@1      Acc@5


        5/5       5.6G      0.246         16        100: 100%|██████████| 1.00/1.00 [00:00<00:00, 2.79batch/s]
***   Best results: [Acc@1: 16.0], [Acc@5: 100.0]



      Epoch   GPU Mem.       Time      Acc@1      Acc@5


        1/5       5.6G     0.2901         16        100: 100%|██████████| 1.00/1.00 [00:00<00:00, 2.52batch/s]
***   Best results: [Acc@1: 16.0], [Acc@5: 100.0]


(16.0, 100.0)